# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import sys
sys.path.append("..")
sys.path.append("../..")

import tensorflow as tf
from keras.losses import BinaryCrossentropy

from models import (
  steganogan_encoder_dense_model,
  steganogan_decoder_dense_model,
)
from keras_steganogan import KerasSteganoGAN

### Constants

In [5]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 2
BATCH_SIZE = 4
MODEL_PATH = '../pretrained_models/steganoGAN_dense.keras'

----

### Call main encode and decode functions (with creating steganographic image and decoding it)

In [6]:
encoder = steganogan_encoder_dense_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)
decoder = steganogan_decoder_dense_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)

steganoGAN = KerasSteganoGAN(
  encoder=encoder,
  decoder=decoder,
  image_shape=IMAGE_SHAPE,
  data_depth=MESSAGE_DEPTH,
  model_path=MODEL_PATH
)

steganoGAN.encode('../../images/testing/input1.png', '../images/dense/output1.png', 'Hello, World! 1111 asdf')
print(steganoGAN.decode('../images/dense/output1.png'))

steganoGAN.encode('../../images/testing/input2.png', '../images/dense/output2.png', 'Hello, World! 2222 asdf')
print(steganoGAN.decode('../images/dense/output2.png'))

steganoGAN.encode('../../images/testing/input3.png', '../images/dense/output3.png', 'Hello, World! 3333 asdf ')
print(steganoGAN.decode('../images/dense/output3.png'))

steganoGAN.encode('../../images/testing/input4.png', '../images/dense/output4.png', 'Hello, World! 4444 asdf')
print(steganoGAN.decode('../images/dense/output4.png'))

Found 111 candidates for message, choosing most common.
Hello, World! 1111 asdf
Found 70 candidates for message, choosing most common.
Hello, World! 2222 asdf
Found 109 candidates for message, choosing most common.
Hello, World! 3333 asdf 
Found 24 candidates for message, choosing most common.
Hello, World! 4444 asdf


### SteganoGAN predict random data with metrics

In [7]:
cover_image = tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS], -1, 1, dtype=tf.float32)
message = tf.cast(tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH], 0, 2, dtype=tf.int32), tf.float32)

stego_img, recovered_msg = steganoGAN.predict([cover_image, message])

print("stego_img min: {0}, max: {1}".format(tf.reduce_min(stego_img), tf.reduce_max(stego_img)))
print("recovered_msg min: {0}, max: {1}".format(tf.reduce_min(recovered_msg), tf.reduce_max(recovered_msg)))

print("BinaryCrossentropy: {0}".format(BinaryCrossentropy(from_logits=False)(message, recovered_msg)))
print("PSNR: {0}".format(tf.reduce_mean(tf.image.psnr(cover_image, stego_img, 1))))
print("SSIM: {0}".format(tf.reduce_mean(tf.image.ssim(cover_image, stego_img, 1))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
stego_img min: -0.9167977571487427, max: 0.879745602607727
recovered_msg min: 0.0, max: 1.0
BinaryCrossentropy: 1.4095714092254639
PSNR: 6.263246536254883
SSIM: 0.19977442920207977
